In [1]:
import numpy as np
import re
import itertools
from collections import Counter
from tensorflow.contrib import learn
import unittests as tests
import tensorflow as tf

# Funcoes Auxiliares (tiradas do paper)

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(positive_data_file, negative_data_file):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]
    # Split by words
    x_text = positive_examples + negative_examples
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in positive_examples]
    negative_labels = [[1, 0] for _ in negative_examples]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]

def batch_iter(data, batch_size, num_batches_per_epoch, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    #pegou todos os dados de treino
    data_size = len(data)
    #meus dados foram dividos em blocos de 64 dados
    #logo, por época 150 (resultado dessa divisao) blocs são usados pra eu acessar todos os dados
    #num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            print('Epoch {:>2}, Sentence Batch {}:  '.format(epoch + 1, batch_num), end='')
            #eu tenho 150 blocos de dados que rodam 200 vezes
            #eu quero pegar só o primeiro bloco
            yield shuffled_data[start_index:end_index]

# Data explore

In [3]:
import random

data,labels = load_data_and_labels('data/rt-polaritydata/rt-polarity.pos','data/rt-polaritydata/rt-polarity.neg')

print (len(data)) #todos os reviews, cada elemento eh um review
print (len(labels)) #todos os sentimentos


10662
10662


In [4]:
(data[0], labels[0]) #tupla primeiro review, no caso eh positivo

("the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal",
 array([0, 1]))

# Pre processing
- Build vocabulary

In [5]:
max_review_size = max([len(x.split(" ")) for x in data])

#Maps documents to sequences of word ids.
#Basicamente cada palavra ja eh mapeada para um id e a sentenca tb
vocab_processor = learn.preprocessing.VocabularyProcessor(max_review_size)
x = np.array(list(vocab_processor.fit_transform(data)))#aqui que ocorre o mapeamento
(x[0], data[0])

(array([ 1,  2,  3,  4,  5,  6,  1,  7,  8,  9, 10, 11, 12, 13, 14,  9, 15,
         5, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0]),
 "the rock is destined to be the 21st century 's new conan and that he 's going to make a splash even greater than arnold schwarzenegger , jean claud van damme or steven segal")

In [6]:
max_review_size

56

- Randomly shuffle data

In [7]:
#Shuffle Data
np.random.seed(10) #for debugging, garante que os numeros aleatorios gerados sempre sejam os mesmos
shuffle_indices = np.random.permutation(np.arange(len(labels)))
shuffle_indices

array([ 7359,  5573, 10180, ...,  1344,  7293,  1289])

In [8]:
# basicamente eu refaço o array de arrays, mas agora cada indice eh oq ta como indice gerado do random
#ex: x[7359] == x_shuffled[0] assim os dois (data, label) mantem o mesmo indice
x_shuffled = x[shuffle_indices] #aqui sao as sentencas ja mapeadas
y_shuffled = labels[shuffle_indices]



 - Train/Validation split

In [9]:
val_percentage = .1
val_sample_index = -1 * int(val_percentage * float(len(labels)))
#TODO - fazer do meu jeito isso aqui
#justamente pra pegar os ultimos enquanto no validation, pega de ordem reversa,
#ex: pegue todos com excessao dos -1066 ultimos
#ou pegue os 1066 primeiros

In [10]:
x_train, x_val = x_shuffled[:val_sample_index], x_shuffled[val_sample_index:]
y_train, y_val = y_shuffled[:val_sample_index], y_shuffled[val_sample_index:]

In [11]:
print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_val)))

Vocabulary Size: 18758
Train/Dev split: 9596/1066


# Build CNN

- Inputs and Labels instances

In [12]:
def neural_net_sentence_input(sentence_size):
    """
    Return a Tensor for a batch of image input
    : sentence_size: Size of the sentence with the biggest len
    : return: Tensor for sentences input.
    Remeber: all sentences were padded to get the max len
    """
    # TODO: Implement Function
    return tf.placeholder(tf.int32, shape=[None,sentence_size],name='input_x')


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, shape=[None,n_classes],name='input_y')


def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, shape=None,name='keep_prob')


"""
UNIT TESTS
"""
tf.reset_default_graph()
tests.test_nn_sentence_inputs(neural_net_sentence_input)
tests.test_nn_label_inputs(neural_net_label_input)
tests.test_nn_keep_prob_inputs(neural_net_keep_prob_input)

Sentence Input Tests Passed.
Label Input Tests Passed.
Keep Prob Tests Passed.


- Embedding Layer

In [13]:
def embedding_creation(x_tensor,vocab_size,embedding_size):
    W = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),name="W")
    embedded_chars = tf.nn.embedding_lookup(W, x_tensor)
    embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
    
    return embedded_chars_expanded


tests.test_embed(embedding_creation)

Tests Passed


- Convolution Layer

In [14]:
def conv2d_maxpool(x_tensor, num_filters, filter_size):
    """
    return: A tensor that represents convolution and max pooling of x_tensor
    """
    embbeding_size = int(x_tensor.shape[2])
    filter_shape = [filter_size,embbeding_size, 1, num_filters]
    
    weights = tf.Variable(tf.truncated_normal(filter_shape,stddev=0.1), name="W")
    bias = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
    """
    Strides controls how the filter convolves around the input
    As we want to go each word per time, everything will have size one
    As we apply conv layers, we could pad the image to preserve the dimension 
    (and try to extract more level features)
    Because we are only dealing with words, this would not be necessary. This is known as narrow convolution
    
    Conv gives us an output of shape [1, sequence_length - filter_size + 1, 1, 1] - There is a formula to discover that
    
    """
    conv = tf.nn.conv2d(x_tensor, weights, strides=[1, 1, 1, 1], padding='VALID')

    conv = tf.nn.bias_add(conv, bias)
    #add non linearity
    h = tf.nn.relu(conv, name="relu")
    sequence_length = int(x_tensor.shape[1])
    conv_output = [1, sequence_length - filter_size + 1, 1, 1]
    
    #Maxpooling over the outputs
    #this will heaturn a tensor of shape [batch_size, 1, 1, num_filters] 
    #which is essencialy a feature vector where the last dimension correspond to features
    #Stride have this size basically because of the same logic applied before
    pooled = tf.nn.max_pool(h, ksize=conv_output,
                            strides=[1, 1, 1, 1],
                            padding='VALID',
                            name='pool') 
    
    return pooled


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_con_pool(conv2d_maxpool)

Tests Passed


- Apply different filters

In [15]:
def apply_conv_filters(x_tensor,filter_sizes,num_filters):
# Create a convolution + maxpool layer for each filter size
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope("conv-maxpool-%s" % filter_size):
            pooled = conv2d_maxpool(x_tensor, num_filters, filter_size)
            pooled_outputs.append(pooled)     
    num_filters_total = num_filters * len(filter_sizes)
    #concat -> sum(Daxis(i)) where Daxis is Dimension axis (in our case is the third one)
    h_pool = tf.concat(pooled_outputs, 3)
    return h_pool

tests.test_apply_filters(apply_conv_filters,conv2d_maxpool)

Tests Passed


- Flatten Layer

The flatten function to change the dimension of x_tensor from a 4-D tensor to a 2-D tensor. The output should be the shape (Batch Size, Flattened Features Size).

In [16]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    #print (tf.contrib.layers.flatten(x_tensor))
    #This is a general flatten function
    flat = x_tensor.shape[1]*x_tensor.shape[2]*x_tensor.shape[3]
    return tf.reshape(x_tensor,[-1,int(flat)])


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_flatten(flatten)

Tests Passed


In [17]:
def output(x_tensor,num_classes):
    num_filters_total = int(x_tensor.shape[1])
    W = tf.Variable(tf.truncated_normal([num_filters_total, num_classes], stddev=0.1), name="W")
    #W = tf.get_variable("W", shape=[num_filters_total, num_classes],initializer=tf.contrib.layers.xavier_initializer())
    b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")

    scores = tf.nn.xw_plus_b(x_tensor, W, b, name="scores")
    return scores
tests.test_output(output)

Tests Passed


- Convolutional Network

In [18]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # TODO: Apply 1, 2, or 3 Convolution and Max Pool layers
    #    Play around with different number of outputs, kernel size and stride
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    
    vocab_size = len(vocab_processor.vocabulary_)
    embed_dim = 128
    embbed_layer = embedding_creation(x,vocab_size,embed_dim)
    
    num_filters = 128
    filter_sizes = [3,4,5]
    conv_layer = apply_conv_filters(embbed_layer,filter_sizes,num_filters)

    

    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    flat_layer = flatten(conv_layer)
    
    with tf.name_scope("dropout"):
        dropout =  tf.nn.dropout(flat_layer, keep_prob)
        
    with tf.name_scope("output"):  
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs) 
        num_classes = 2
        output_layer = output(dropout, num_classes)

    
    return output_layer

Build The Neural Network

In [25]:
# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_sentence_input(56) #sequence_length
y = neural_net_label_input(2) #positive or negative
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
#cost =  cost + l2_reg_lambda * l2_loss
optimizer = tf.train.AdamOptimizer().minimize(cost)
#optimizer = tf.train.AdamOptimizer(1e-3)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


"""
UNIT TEST
"""
tests.test_conv_net(conv_net)

Neural Network Built!


# Training Process

In [26]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    """
    Optimize the session on a batch of images and labels
    : session: Current TensorFlow session
    : optimizer: TensorFlow optimizer function
    : keep_probability: keep probability
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    """
    
    
    session.run(optimizer, feed_dict={
            x: feature_batch,
            y: label_batch,
            keep_prob: keep_probability})


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_train_nn(train_neural_network)

Tests Passed


# Print statistics

In [30]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    loss = session.run(cost, feed_dict={
            x: feature_batch,
            y: label_batch,
            keep_prob: 1.})
    
    valid_acc = session.run(accuracy, feed_dict={
        x: x_val,
        y: y_val,
        keep_prob: 1.})
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss,valid_acc))

# Hyperparameters

In [31]:
epochs = 27
batch_size = 64
keep_probability =  0.5
l2_reg_lambda = 0.0
num_batches_per_epoch = int((len(list(zip(x_train, y_train)))-1)/batch_size) + 1

# Training on a Single Batch

In [32]:
print('Checking the Training on a Single Batch...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    # Generate single batches
    batches = batch_iter(list(zip(x_train, y_train)), batch_size, 1, epochs, shuffle=False)
    # Training cycle
    for batch in batches:
        batch_features, batch_labels = zip(*batch)
        train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
        print_stats(sess, batch_features, batch_labels, cost, accuracy)

Checking the Training on a Single Batch...
Epoch  1, Sentence Batch 0:  Loss:     0.7206 Validation Accuracy: 0.512195
Epoch  2, Sentence Batch 0:  Loss:     0.4822 Validation Accuracy: 0.526266
Epoch  3, Sentence Batch 0:  Loss:     0.3022 Validation Accuracy: 0.522514
Epoch  4, Sentence Batch 0:  Loss:     0.1971 Validation Accuracy: 0.530957
Epoch  5, Sentence Batch 0:  Loss:     0.1215 Validation Accuracy: 0.525328
Epoch  6, Sentence Batch 0:  Loss:     0.0797 Validation Accuracy: 0.521576
Epoch  7, Sentence Batch 0:  Loss:     0.0565 Validation Accuracy: 0.517824
Epoch  8, Sentence Batch 0:  Loss:     0.0387 Validation Accuracy: 0.516886
Epoch  9, Sentence Batch 0:  Loss:     0.0285 Validation Accuracy: 0.515947
Epoch 10, Sentence Batch 0:  Loss:     0.0214 Validation Accuracy: 0.514071
Epoch 11, Sentence Batch 0:  Loss:     0.0157 Validation Accuracy: 0.516886
Epoch 12, Sentence Batch 0:  Loss:     0.0116 Validation Accuracy: 0.519700
Epoch 13, Sentence Batch 0:  Loss:     0.0087

KeyboardInterrupt: 

In [31]:
print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    # Generate single batches
    batches = batch_iter(list(zip(x_train, y_train)), batch_size, num_batches_per_epoch, epochs, shuffle=True)
    # Training cycle
    for batch in batches:
        batch_features, batch_labels = zip(*batch)
        train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
        print_stats(sess, batch_features, batch_labels, cost, accuracy)

Training...
Epoch  1, Sentence Batch 0:  Loss:     3.2484 Validation Accuracy: 0.483114
Epoch  1, Sentence Batch 1:  Loss:     2.8926 Validation Accuracy: 0.492495
Epoch  1, Sentence Batch 2:  Loss:     1.0546 Validation Accuracy: 0.472795
Epoch  1, Sentence Batch 3:  Loss:     1.0607 Validation Accuracy: 0.492495
Epoch  1, Sentence Batch 4:  Loss:     1.6892 Validation Accuracy: 0.511257
Epoch  1, Sentence Batch 5:  Loss:     2.4686 Validation Accuracy: 0.514071
Epoch  1, Sentence Batch 6:  Loss:     2.8299 Validation Accuracy: 0.515009
Epoch  1, Sentence Batch 7:  Loss:     2.1434 Validation Accuracy: 0.514071
Epoch  1, Sentence Batch 8:  Loss:     2.1150 Validation Accuracy: 0.516886
Epoch  1, Sentence Batch 9:  Loss:     1.7146 Validation Accuracy: 0.515947
Epoch  1, Sentence Batch 10:  Loss:     1.6089 Validation Accuracy: 0.516886
Epoch  1, Sentence Batch 11:  Loss:     1.1082 Validation Accuracy: 0.498124
Epoch  1, Sentence Batch 12:  Loss:     1.1699 Validation Accuracy: 0.4990

Epoch  1, Sentence Batch 106:  Loss:     0.7457 Validation Accuracy: 0.577861
Epoch  1, Sentence Batch 107:  Loss:     0.6778 Validation Accuracy: 0.580675
Epoch  1, Sentence Batch 108:  Loss:     0.5241 Validation Accuracy: 0.579737
Epoch  1, Sentence Batch 109:  Loss:     0.8042 Validation Accuracy: 0.585366
Epoch  1, Sentence Batch 110:  Loss:     0.6573 Validation Accuracy: 0.575985
Epoch  1, Sentence Batch 111:  Loss:     0.8050 Validation Accuracy: 0.575047
Epoch  1, Sentence Batch 112:  Loss:     0.8991 Validation Accuracy: 0.551595
Epoch  1, Sentence Batch 113:  Loss:     0.8471 Validation Accuracy: 0.545966
Epoch  1, Sentence Batch 114:  Loss:     0.8940 Validation Accuracy: 0.545966
Epoch  1, Sentence Batch 115:  Loss:     0.7592 Validation Accuracy: 0.544090
Epoch  1, Sentence Batch 116:  Loss:     0.8480 Validation Accuracy: 0.556285
Epoch  1, Sentence Batch 117:  Loss:     0.6886 Validation Accuracy: 0.562852
Epoch  1, Sentence Batch 118:  Loss:     0.6526 Validation Accur

Epoch  2, Sentence Batch 62:  Loss:     0.5712 Validation Accuracy: 0.606004
Epoch  2, Sentence Batch 63:  Loss:     0.4951 Validation Accuracy: 0.602251
Epoch  2, Sentence Batch 64:  Loss:     0.4636 Validation Accuracy: 0.600375
Epoch  2, Sentence Batch 65:  Loss:     0.4813 Validation Accuracy: 0.594747
Epoch  2, Sentence Batch 66:  Loss:     0.4527 Validation Accuracy: 0.583490
Epoch  2, Sentence Batch 67:  Loss:     0.4765 Validation Accuracy: 0.576923
Epoch  2, Sentence Batch 68:  Loss:     0.5462 Validation Accuracy: 0.573171
Epoch  2, Sentence Batch 69:  Loss:     0.6520 Validation Accuracy: 0.571295
Epoch  2, Sentence Batch 70:  Loss:     0.4923 Validation Accuracy: 0.578799
Epoch  2, Sentence Batch 71:  Loss:     0.4687 Validation Accuracy: 0.583490
Epoch  2, Sentence Batch 72:  Loss:     0.4732 Validation Accuracy: 0.581613
Epoch  2, Sentence Batch 73:  Loss:     0.4806 Validation Accuracy: 0.582552
Epoch  2, Sentence Batch 74:  Loss:     0.4036 Validation Accuracy: 0.577861

Epoch  3, Sentence Batch 18:  Loss:     0.4532 Validation Accuracy: 0.613508
Epoch  3, Sentence Batch 19:  Loss:     0.4160 Validation Accuracy: 0.618199
Epoch  3, Sentence Batch 20:  Loss:     0.3495 Validation Accuracy: 0.616323
Epoch  3, Sentence Batch 21:  Loss:     0.3856 Validation Accuracy: 0.616323
Epoch  3, Sentence Batch 22:  Loss:     0.4986 Validation Accuracy: 0.617261
Epoch  3, Sentence Batch 23:  Loss:     0.3659 Validation Accuracy: 0.615385
Epoch  3, Sentence Batch 24:  Loss:     0.3767 Validation Accuracy: 0.624765
Epoch  3, Sentence Batch 25:  Loss:     0.3584 Validation Accuracy: 0.619137
Epoch  3, Sentence Batch 26:  Loss:     0.3566 Validation Accuracy: 0.629456
Epoch  3, Sentence Batch 27:  Loss:     0.3199 Validation Accuracy: 0.621013
Epoch  3, Sentence Batch 28:  Loss:     0.3018 Validation Accuracy: 0.629456
Epoch  3, Sentence Batch 29:  Loss:     0.4891 Validation Accuracy: 0.632270
Epoch  3, Sentence Batch 30:  Loss:     0.4418 Validation Accuracy: 0.613508

Epoch  3, Sentence Batch 124:  Loss:     0.4873 Validation Accuracy: 0.609756
Epoch  3, Sentence Batch 125:  Loss:     0.4659 Validation Accuracy: 0.626642
Epoch  3, Sentence Batch 126:  Loss:     0.3609 Validation Accuracy: 0.636961
Epoch  3, Sentence Batch 127:  Loss:     0.5243 Validation Accuracy: 0.637899
Epoch  3, Sentence Batch 128:  Loss:     0.4267 Validation Accuracy: 0.636961
Epoch  3, Sentence Batch 129:  Loss:     0.5059 Validation Accuracy: 0.632270
Epoch  3, Sentence Batch 130:  Loss:     0.4447 Validation Accuracy: 0.634146
Epoch  3, Sentence Batch 131:  Loss:     0.3805 Validation Accuracy: 0.634146
Epoch  3, Sentence Batch 132:  Loss:     0.3839 Validation Accuracy: 0.634146
Epoch  3, Sentence Batch 133:  Loss:     0.3997 Validation Accuracy: 0.632270
Epoch  3, Sentence Batch 134:  Loss:     0.3635 Validation Accuracy: 0.632270
Epoch  3, Sentence Batch 135:  Loss:     0.4344 Validation Accuracy: 0.634146
Epoch  3, Sentence Batch 136:  Loss:     0.4157 Validation Accur

...

Epoch 25, Sentence Batch 6:  Loss:     0.0013 Validation Accuracy: 0.726079
Epoch 25, Sentence Batch 7:  Loss:     0.0013 Validation Accuracy: 0.726079
Epoch 25, Sentence Batch 8:  Loss:     0.0018 Validation Accuracy: 0.727017
Epoch 25, Sentence Batch 9:  Loss:     0.0014 Validation Accuracy: 0.729831
Epoch 25, Sentence Batch 10:  Loss:     0.0012 Validation Accuracy: 0.730769
Epoch 25, Sentence Batch 11:  Loss:     0.0031 Validation Accuracy: 0.729831
Epoch 25, Sentence Batch 12:  Loss:     0.0016 Validation Accuracy: 0.728893
Epoch 25, Sentence Batch 13:  Loss:     0.0015 Validation Accuracy: 0.729831
Epoch 25, Sentence Batch 14:  Loss:     0.0031 Validation Accuracy: 0.729831
Epoch 25, Sentence Batch 15:  Loss:     0.0004 Validation Accuracy: 0.728893
Epoch 25, Sentence Batch 16:  Loss:     0.0013 Validation Accuracy: 0.729831
Epoch 25, Sentence Batch 17:  Loss:     0.0006 Validation Accuracy: 0.728893
Epoch 25, Sentence Batch 18:  Loss:     0.0013 Validation Accuracy: 0.727955
Epo

Epoch 25, Sentence Batch 112:  Loss:     0.0017 Validation Accuracy: 0.724203
Epoch 25, Sentence Batch 113:  Loss:     0.0012 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 114:  Loss:     0.0017 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 115:  Loss:     0.0015 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 116:  Loss:     0.0011 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 117:  Loss:     0.0005 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 118:  Loss:     0.0020 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 119:  Loss:     0.0003 Validation Accuracy: 0.723265
Epoch 25, Sentence Batch 120:  Loss:     0.0008 Validation Accuracy: 0.722326
Epoch 25, Sentence Batch 121:  Loss:     0.0006 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 122:  Loss:     0.0010 Validation Accuracy: 0.721388
Epoch 25, Sentence Batch 123:  Loss:     0.0007 Validation Accuracy: 0.722326
Epoch 25, Sentence Batch 124:  Loss:     0.0014 Validation Accur

Epoch 26, Sentence Batch 68:  Loss:     0.0027 Validation Accuracy: 0.730769
Epoch 26, Sentence Batch 69:  Loss:     0.0008 Validation Accuracy: 0.730769
Epoch 26, Sentence Batch 70:  Loss:     0.0008 Validation Accuracy: 0.730769
Epoch 26, Sentence Batch 71:  Loss:     0.0059 Validation Accuracy: 0.727955
Epoch 26, Sentence Batch 72:  Loss:     0.0030 Validation Accuracy: 0.726079
Epoch 26, Sentence Batch 73:  Loss:     0.0012 Validation Accuracy: 0.724203
Epoch 26, Sentence Batch 74:  Loss:     0.0009 Validation Accuracy: 0.723265
Epoch 26, Sentence Batch 75:  Loss:     0.0036 Validation Accuracy: 0.725141
Epoch 26, Sentence Batch 76:  Loss:     0.0025 Validation Accuracy: 0.723265
Epoch 26, Sentence Batch 77:  Loss:     0.0034 Validation Accuracy: 0.722326
Epoch 26, Sentence Batch 78:  Loss:     0.0011 Validation Accuracy: 0.719512
Epoch 26, Sentence Batch 79:  Loss:     0.0007 Validation Accuracy: 0.722326
Epoch 26, Sentence Batch 80:  Loss:     0.0009 Validation Accuracy: 0.722326

Epoch 27, Sentence Batch 24:  Loss:     0.0013 Validation Accuracy: 0.724203
Epoch 27, Sentence Batch 25:  Loss:     0.0009 Validation Accuracy: 0.726079
Epoch 27, Sentence Batch 26:  Loss:     0.0007 Validation Accuracy: 0.725141
Epoch 27, Sentence Batch 27:  Loss:     0.0003 Validation Accuracy: 0.723265
Epoch 27, Sentence Batch 28:  Loss:     0.0008 Validation Accuracy: 0.723265
Epoch 27, Sentence Batch 29:  Loss:     0.0007 Validation Accuracy: 0.725141
Epoch 27, Sentence Batch 30:  Loss:     0.0009 Validation Accuracy: 0.723265
Epoch 27, Sentence Batch 31:  Loss:     0.0020 Validation Accuracy: 0.722326
Epoch 27, Sentence Batch 32:  Loss:     0.0015 Validation Accuracy: 0.722326
Epoch 27, Sentence Batch 33:  Loss:     0.0004 Validation Accuracy: 0.721388
Epoch 27, Sentence Batch 34:  Loss:     0.0012 Validation Accuracy: 0.723265
Epoch 27, Sentence Batch 35:  Loss:     0.0010 Validation Accuracy: 0.724203
Epoch 27, Sentence Batch 36:  

KeyboardInterrupt: 

# Batch generation

In [ ]:
print_stats(sess, batch_features, batch_labels, cost, accuracy)
        train_step(x_batch, y_batch)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % FLAGS.evaluate_every == 0:
            print("\nEvaluation:")
            dev_step(x_dev, y_dev, writer=dev_summary_writer)
            print("")
        if current_step % FLAGS.checkpoint_every == 0:
            path = saver.save(sess, checkpoint_prefix, global_step=current_step)
            print("Saved model checkpoint to {}\n".format(path))